NER - это задача идентификации и классификации именованных сущностей в тексте на предопределенные категории.

**Основные типы сущностей**:
 - `PER` - персоны (Иван Иванов)
 - `LOC` - локации (Москва, река Волга)
 - `ORG` - организации (Яндекс, МТС)
 - `DATE` - даты (5 мая 2023 года)

**Пример разметки**:
 ```
 [Иван Петров]PER работает в [Москве]LOC с [2020 года]DATE.
 ```

Архитектура BiLSTM для NER более удобна
```
Embedding -> BiLSTM -> Linear -> CRF
```
**Особенности**:
- Двунаправленные LSTM для учета контекста
- Char-level эмбеддинги для работы с морфологией
- CRF-слой для учета зависимостей между тегами

Разметка данных и метрики

**BIO-схема**:
```
Токен    Метка
--------------
Apple    B-ORG
Inc.     I-ORG
in       O
Paris    B-LOC
```
**В задании предстит использовать базовый lstm**

#### Метрики качества в NER

Классические метрики из задач классификации и извлечения информации здесь требуют особого подхода, учитывая последовательный (токенизированный) характер разметки.

**Часто используются следующие метрики:**

1. **Precision (точность):** Какую долю найденных вами сущностей реально существуют в разметке? 
   ```math 
   Precision = \frac{\text{Число корректно найденных сущностей}}{\text{Число найденных сущностей всего}}
   ```

3. **Recall (полнота):** Какую долю от всех размеченных сущностей модель смогла найти?  
   ```math 
   Recall = \frac{\text{Число корректно найденных сущностей}}{\text{Число всех эталонных сущностей}}
   ```

4. **F1-score:**  
   Среднее гармоническое Precision и Recall:
   ```math
   F_1 = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}
   ```

5. **Exact match (точное совпадение):**  
   Сущность угадана полностью (границы и класс совпадают!)

6. **Partial match (частичное совпадение):** Границы сущности пересекаются, но не совпадают точно.

7. **Missing (пропущенные):** Сущности, которые есть в разметке, но не найдены моделью (Recall).

**В NER** сущность считается предсказанной верно, если полностью совпадают: начало, конец и тип сущности (для Exact match).

- **Exact match** — самое строгое требование: границы entity и класс должны совпасть.
- **Partial match** — хоть частично entity покрыта (например, модель угадала "Иван Иванов", а в разметке было "Иванов").

**Пример:**
Текст:  
```
"Иван Иванов работает в Яндексе"
```
Разметка:  
PER: "Иван Иванов"  
ORG: "Яндексе"

- Если модель предскажет PER: "Иван", это будет partial!
- Если ORG: "Яндекс" (без "е") — тоже partial.

#### Как считать эти метрики

Если ваши сущности представлены как интервалы (start, end, type), то:
- **Exact:** start, end, type полностью совпадают.
- **Partial:** есть хоть какое-то пересечение.
- **Missing:** сущность есть в эталонной разметке, но модель её не предсказала не в каком виде.

**ВАЖНО:**  
В NER необходимо считать метрики именно на уровне сущностей (span-level), а не токенов.


#### Применение в задании

- Для вашего задания важно реализовать измерение **exact, partial, missing** (и обычно precision, recall, f1) для сравнения своей и предобученной модели (например Natasha).
- Кроме численных метрик желательно посмотреть визуальные примеры (с помощью, например, функции `show_box_markup`).


#### Итог

- NER ищет и классифицирует сущности в тексте.
- Оценивается обычно через метрики: Precision, Recall, F1, Exact match, Partial match, Missing entities.
- В отличие от токен-классификации важны именно отрезки (интервалы) сущностей.
- Для реальных приложений максимализируется обычно **F1** по **exact**.

---

### Структура коллекции
```
collection/
  ├── папка_123/         # Произвольное имя
  │   ├── text.txt
  │   └── anno.markup.xml
  ├── document_xyz/      # Другое произвольное имя
  │   ├── text.txt
  │   └── anno.markup.xml
  └── ...
```


### Пример разметки сущности

```xml
<entry>
<id>1</id>
<offset>218</offset>
<length>13</length>
<class>AAA_Estimate_Person</class>
<attribute>
  <name>Canonical</name>
  <value>РОБЕРТ МУГАБЕ</value>
</attribute>
</entry>
```

#### Детальный разбор:

- **id** — уникальный идентификатор этой сущности в разметке (в данном случае: 1).
- **offset** — позиция (смещение) первого символа сущности в исходном тексте, считая от начала строки; здесь 218.
- **length** — длина сущности в символах; здесь 13.
- **class** — тип сущности; здесь AAA_Estimate_Person (вероятно, аналог типа *PERSON* — то есть человек).
- **attribute** — дополнительная информация о сущности. - Вложенные теги:  
    - **name**: Название атрибута (например, "Canonical" — каноническое имя).
    - **value**: Значение атрибута — здесь "РОБЕРТ МУГАБЕ" (имя человека в каноническом виде).

#### Как это работает в NER

- В тексте, начиная с позиции 218 и длиной 13 символов, встречается имя, соответствующее классу "человек" (PERSON).
- Канонический вид записи указывается явно — иногда текст может содержать склонённые формы, сокращения, альтернативные написания, а каноническая форма помогает их нормализовать.

#### Визуализация

Если бы это был фрагмент текста, например:

```
... президент страны Роберт Мугабе выступил с обращением ...
                   ^ начало на 218 позиции, далее 13 символов -> "Роберт Мугабе"
```

#### Связь с NER

Такой формат часто используют для хранения разметки для задачи NER — для дальнейшего сравнения с выходами моделей: проверяют совпадают ли найденные моделью сущности по классам, началу/концу, и сопоставляют ли правильные канонические имена.

--------------


В данном ноутбуке представлены примеры структуры данных, шаблоны основных функций и классов.

Модуль `seqeval` — это специализированная библиотека на Python для оценки качества моделей, решающих задачи извлечения последовательностей сущностей (Sequence Labeling), такие как Named Entity Recognition (NER), Chunking, POS-tagging и др.

In [25]:
! pip install seqeval

In [26]:
import os
import xml.etree.ElementTree as ET
from glob import glob
from IPython.display import display, HTML
import json
from collections import defaultdict
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F

import numpy as np
from sklearn.metrics import classification_report

### 1. Поиск и загрузка документов

In [27]:
def load_collection(collection_path):
    """загрузка коллекции документов с текстами и разметкой
    
    параметры:
        collection_path: путь к корневой папке с данными
    
    возвращает:
        список документов в формате {'text': текст, 'xml_tree': xml-дерево}
    """
    documents = []
    # рекурсивный поиск всех файлов разметки
    for xml_path in glob(os.path.join(collection_path, '**/anno.markup.xml'), recursive=True):
        dir_path = os.path.dirname(xml_path)
        text_path = os.path.join(dir_path, 'text.txt')
        
        if os.path.exists(text_path):
            try:
                # чтение текста в кодировке utf-8
                # with open(text_path, 'r', encoding='utf-8') as f:
                with open(text_path, 'r', encoding='cp1251') as f:
                    text = f.read()
                
                # парсинг xml-разметки
                tree = ET.parse(xml_path)
                documents.append({
                    'text': text,
                    'xml_tree': tree
                })
            except Exception as e:
                print(f"ошибка при загрузке {dir_path}: {e}")
    
    return documents

# загрузка данных
collection_path = 'Persons-1000/collection'
documents = load_collection(collection_path)
print(f"Успешно загружено документов: {len(documents)}")


Успешно загружено документов: 1000


In [28]:
documents

[{'text': 'Нового руководителя подмосковной прокуратуры нашли в столице\n\n\nИсполнять обязанности руководителя прокуратуры Московской области с сегодняшнего дня поручено Алексею Захарову, который до сих пор работал заместителем прокурора Москвы. На этом посту он заменит ушедшего в отставку Александра Аникина.\n\nПо данным газеты "Коммерсантъ", А.Аникин, который занял пост руководителя прокуратуры Московской области после знаменитого "игорного дела", за два года заменил 28 городских, районных и специализированных прокуроров, проведя внеплановую переаттестацию сотрудников.\n\nОднако проблемы возникли и у самого А.Аникина. По данным издания, у него были разногласия с двумя заместителями генерального прокурора - Виктором Гринем и Владимиром Малиновским. В результате первую попытку уйти в отставку А.Аникин предпринял еще весной, но его просьбу отклонил Юрий Чайка. Теперь же просьба прокурора Подмосковья удовлетворена. Он написал заявление и ушел в 45-дневный отпуск.\n\nЧто касается А.Захар

In [29]:
documents[0]

{'text': 'Нового руководителя подмосковной прокуратуры нашли в столице\n\n\nИсполнять обязанности руководителя прокуратуры Московской области с сегодняшнего дня поручено Алексею Захарову, который до сих пор работал заместителем прокурора Москвы. На этом посту он заменит ушедшего в отставку Александра Аникина.\n\nПо данным газеты "Коммерсантъ", А.Аникин, который занял пост руководителя прокуратуры Московской области после знаменитого "игорного дела", за два года заменил 28 городских, районных и специализированных прокуроров, проведя внеплановую переаттестацию сотрудников.\n\nОднако проблемы возникли и у самого А.Аникина. По данным издания, у него были разногласия с двумя заместителями генерального прокурора - Виктором Гринем и Владимиром Малиновским. В результате первую попытку уйти в отставку А.Аникин предпринял еще весной, но его просьбу отклонил Юрий Чайка. Теперь же просьба прокурора Подмосковья удовлетворена. Он написал заявление и ушел в 45-дневный отпуск.\n\nЧто касается А.Захаро

In [30]:
def parse_annotations(xml_tree):
    """извлечение аннотаций из xml-дерева
    
    параметры:
        xml_tree: дерево xml с разметкой
    
    возвращает:
        список аннотаций в формате {'offset': позиция, 'length': длина, 'text': текст}
    """
    annotations = []
    for entry in xml_tree.findall('entry'):
        try:
            # извлечение данных из xml-узла
            annotations.append({
                'offset': int(entry.find('offset').text), # позиция начала
                'length': int(entry.find('length').text), # длина сущности
                'text': entry.find('attribute/value').text.strip() # текст сущности
            })
        except Exception as e:
            print(f"ошибка при парсинге аннотации: {e}")
    return annotations


# пример для первого документа
sample_doc = documents[0]
annotations = parse_annotations(sample_doc['xml_tree'])
print(f"Найдено аннотаций в первом документе: {len(annotations)}")

Найдено аннотаций в первом документе: 11


In [31]:
annotations

[{'offset': 160, 'length': 16, 'text': 'АЛЕКСЕЙ ЗАХАРОВ'},
 {'offset': 281, 'length': 18, 'text': 'АЛЕКСАНДР АНИКИН'},
 {'offset': 336, 'length': 8, 'text': 'А АНИКИН'},
 {'offset': 607, 'length': 9, 'text': 'А АНИКИН'},
 {'offset': 708, 'length': 15, 'text': 'ВИКТОР ГРИНЬ'},
 {'offset': 726, 'length': 22, 'text': 'ВЛАДИМИР МАЛИНОВСКИЙ'},
 {'offset': 794, 'length': 8, 'text': 'А АНИКИН'},
 {'offset': 850, 'length': 10, 'text': 'ЮРИЙ ЧАЙКА'},
 {'offset': 982, 'length': 10, 'text': 'А ЗАХАРОВ'},
 {'offset': 1171, 'length': 9, 'text': 'А ЗАХАРОВ'},
 {'offset': 1264, 'length': 9, 'text': 'А ЗАХАРОВ'}]

In [32]:
def build_vocabularies(documents):
    """создание словарей для слов и символов
    
    параметры:
        documents: список документов для построения словарей
    
    возвращает:
        word_vocab: словарь слов {слово: id}
        char_vocab: словарь символов {символ: id}
        tag_vocab: словарь тегов {'O': 0, 'B-PER': 1, ...}
    """
    # словари с автоматической нумерацией
    word_vocab = defaultdict(lambda: len(word_vocab))
    char_vocab = defaultdict(lambda: len(char_vocab))
    
    # базовые теги для ner
    tag_vocab = {'O': 0, 'B-PER': 1, 'I-PER': 2}
    
    # специальные токены
    _ = word_vocab['<PAD>'] # токен заполнения
    _ = word_vocab['<UNK>'] # неизвестное слово
    _ = char_vocab['<PAD>'] # заполнение для символов
    _ = char_vocab['<UNK>'] # неизвестный символ
    
    # заполнение словарей
    for doc in documents:
        for word in doc['text'].split():
            # добавление слова в нижнем регистре
            _ = word_vocab[word.lower()]
            # добавление всех символов слова
            for char in word:
                _ = char_vocab[char]
                
    return word_vocab, char_vocab, tag_vocab

# создаем словари
word_vocab, char_vocab, tag_vocab = build_vocabularies(documents)
print(f"Размер словаря слов: {len(word_vocab)}")
print(f"Размер словаря символов: {len(char_vocab)}")

Размер словаря слов: 44159
Размер словаря символов: 168


In [33]:
# id2tag и имена классов
id_to_tag = {id: tag for tag, id in tag_vocab.items()}
target_names = [id_to_tag[i] for i in range(len(tag_vocab))]

Подготовим сырые тексты и разметку в формат, понятный нейросетевой модели

In [34]:
class NERDataset(Dataset):
    def __init__(self, documents, word_vocab, char_vocab, tag_vocab, max_len=128, max_word_len=25):
        """датасет для преобразования текстов в тензоры для ner.
        
        параметры:
            documents: список документов с текстами и разметкой
            word_vocab: словарь {слово: id} для слов
            char_vocab: словарь {символ: id} для символов
            tag_vocab: словарь {тег: id} для меток (b-per, i-per, o)
            max_len: максимальная длина последовательности (в токенах)
            max_word_len: максимальная длина слова (в символах)
        """
        self.documents = documents
        self.word_vocab = word_vocab
        self.char_vocab = char_vocab
        self.tag_vocab = tag_vocab
        self.max_len = max_len
        self.max_word_len = max_word_len
        
    def __len__(self):
        """возвращает количество документов в датасете"""
        return len(self.documents)
    
    def __getitem__(self, idx):
        """возвращает один элемент датасета по индексу"""
        doc = self.documents[idx]
        text = doc['text']
        annotations = parse_annotations(doc['xml_tree'])
        
        # разбиваем текст на токены (с ограничением по max_len)
        tokens = text.split()[:self.max_len]
        
        # создаем тензоры для хранения данных
        word_ids = torch.zeros(self.max_len, dtype=torch.long)  # id слов
        char_ids = torch.zeros((self.max_len, self.max_word_len), dtype=torch.long)  # id символов
        tag_ids = torch.zeros(self.max_len, dtype=torch.long)  # id меток
        
        # заполняем тензоры
        for i, token in enumerate(tokens):
            # преобразуем слово в id (если нет в словаре - используем <unk>)
            word_ids[i] = self.word_vocab.get(token.lower(), self.word_vocab['<unk>'])
            
            # преобразуем символы слова в id
            chars = [
                self.char_vocab.get(c, self.char_vocab['<unk>']) 
                for c in token[:self.max_word_len]
            ]
            # добавляем паддинг до max_word_len
            chars += [self.char_vocab['<pad>']] * (self.max_word_len - len(chars))
            char_ids[i] = torch.tensor(chars, dtype=torch.long)
            
            # по умолчанию ставим метку 'O' (не сущность)
            tag_ids[i] = self.tag_vocab['O']
        
        # обновляем метки на основе аннотаций
        for ann in annotations:
            start = ann['offset']
            end = start + ann['length']
            ann_text = text[start:end]
            
            # ищем токены, попадающие в аннотацию
            pos = 0
            for i, token in enumerate(tokens):
                token_pos = text.find(token, pos)
                if token_pos == -1:  # токен не найден
                    continue
                
                token_end = token_pos + len(token)
                # проверяем пересечение с аннотацией
                if token_pos >= end or token_end <= start:
                    pos = token_end
                    continue
                
                # определяем тип метки:
                if token_pos >= start and token_end <= end:
                    # токен полностью внутри аннотации
                    if token_pos == start:  # начало сущности
                        tag_ids[i] = self.tag_vocab['B-PER']
                    else:  # продолжение сущности
                        tag_ids[i] = self.tag_vocab['I-PER']
                else:
                    # частичное пересечение - отмечаем как начало
                    tag_ids[i] = self.tag_vocab['B-PER']
                
                pos = token_end
        
        return {
            'word_ids': word_ids,  # тензор id слов (max_len)
            'char_ids': char_ids,  # тензор id символов (max_len, max_word_len)
            'tag_ids': tag_ids,    # тензор id меток (max_len)
            'text': text,          # исходный текст (для отладки)
            'tokens': tokens,      # список токенов (для отладки)
            'annotations': annotations  # исходные аннотации (для отладки)
        }

### Базовая LSTM для NER

In [35]:
class LSTM_NER(nn.Module):
    def __init__(self, vocab_size, char_vocab_size, num_tags,
                 word_embed_dim=100, char_embed_dim=30, hidden_dim=128, bi=False, concat=True):
        """однонаправленная lstm модель для извлечения сущностей.
        
        параметры:
            vocab_size: размер словаря слов
            char_vocab_size: размер словаря символов
            num_tags: число классов тегов (b-per, i-per, o)
            word_embed_dim: размерность эмбеддингов слов
            char_embed_dim: размерность эмбеддингов символов
            hidden_dim: размер скрытого состояния lstm
        """
        super().__init__()

        self.bi = bi
        self.concat = concat
        
        # эмбеддинги слов (batch_size, seq_len) -> (batch_size, seq_len, word_embed_dim)
        self.word_embed = nn.Embedding(vocab_size, word_embed_dim)
        
        # эмбеддинги символов (batch_size, seq_len, max_word_len) -> (batch_size, seq_len, char_embed_dim)
        self.char_embed = nn.Embedding(char_vocab_size, char_embed_dim)
        # однонаправленная lstm для символов
        self.char_lstm = nn.LSTM(char_embed_dim, hidden_dim//2, bidirectional=bi)
        
        # основная lstm (однонаправленная) hidden_dim * 2 (если bidirectional=True)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, bidirectional=bi)

        if not self.concat:
            # (batch_size, seq_len, word_embed_dim) -> (batch_size, seq_len, hidden_dim)
            self.word_linear = nn.Linear(word_embed_dim, hidden_dim)
            if self.bi:
                # (batch_size, seq_len, hidden_dim) -> (batch_size, seq_len, hidden_dim)
                self.char_linear = nn.Linear(hidden_dim, hidden_dim)
            else:
                # (batch_size, seq_len, hidden_dim//2) -> (batch_size, seq_len, hidden_dim)
                self.char_linear = nn.Linear(hidden_dim//2, hidden_dim)
        
        if self.bi:      
            # линейный слой для объединения признаков (вместо конкатенации)
            # (batch_size, seq_len, word_embed_dim + hidden_dim) -> (batch_size, seq_len, hidden_dim)
            self.linear = nn.Linear(word_embed_dim + hidden_dim, hidden_dim)
            # классификатор (batch_size, seq_len, hidden_dim * 2) -> (batch_size, seq_len, num_tags)
            self.classifier = nn.Linear(hidden_dim * 2, num_tags)
        else:
            # линейный слой для объединения признаков (вместо конкатенации)
            # (batch_size, seq_len, word_embed_dim + hidden_dim//2) -> (batch_size, seq_len, hidden_dim)
            self.linear = nn.Linear(word_embed_dim + hidden_dim//2, hidden_dim)
            # классификатор (batch_size, seq_len, hidden_dim) -> (batch_size, seq_len, num_tags)
            self.classifier = nn.Linear(hidden_dim, num_tags)

    def forward(self, word_ids, char_ids):
        # размеры:
        # word_ids: (batch_size, seq_len)
        # char_ids: (batch_size, seq_len, max_word_len)
        
        # получаем эмбеддинги слов
        word_emb = self.word_embed(word_ids)
        
        # обрабатываем символы
        batch_size, seq_len, max_word_len = char_ids.size()
        char_emb = self.char_embed(char_ids.view(-1, max_word_len))  # (batch_size*seq_len, max_word_len, char_embed_dim)
        char_out, _ = self.char_lstm(char_emb)  # (batch_size*seq_len, max_word_len, hidden_dim//2)
        char_features = char_out[:, -1, :].view(batch_size, seq_len, -1)  # (batch_size, seq_len, hidden_dim//2)
        
        if self.concat:
            #  объединяем признаки через линейный слой + relu
            combined = torch.cat([word_emb, char_features], dim=-1) #(word_embed_dim + hidden_dim//2) или (word_embed_dim + hidden_dim)
            features = torch.relu(self.linear(combined))  # (batch_size, seq_len, hidden_dim)
        else:
            # суммирование признаков 
            word_features = torch.relu(self.word_linear(word_emb))  # (batch_size, seq_len, hidden_dim)
            char_features = torch.relu(self.char_linear(char_features))  # (batch_size, seq_len, hidden_dim)
            features = word_features + char_features  # (batch_size, seq_len, hidden_dim)
        
        # основная lstm
        lstm_out, _ = self.lstm(features)  # (batch_size, seq_len, hidden_dim) для bidirectional (batch_size, seq_len, hidden_dim * 2)
        
        # классификация
        logits = self.classifier(lstm_out)  # (batch_size, seq_len, num_tags)
        return logits

In [36]:
from torch.utils.data import DataLoader
import copy
import datetime
import traceback
from torch.nn.utils.rnn import pad_sequence

def custom_collate_fn(batch):
    word_ids = pad_sequence([item['word_ids'] for item in batch], batch_first=True, padding_value=word_vocab['<PAD>'])
    char_ids = pad_sequence([item['char_ids'] for item in batch], batch_first=True, padding_value=char_vocab['<PAD>'])
    tag_ids = pad_sequence([item['tag_ids'] for item in batch], batch_first=True, padding_value=tag_vocab['O'])
    tokens = [item['tokens'] for item in batch]
    return word_ids, char_ids, tag_ids, tokens

def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]
    raise ValueError('Недопустимый тип данных {}'.format(type(data)))

def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=5, batch_size=64,
                    device=None, early_stopping_patience=10,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    lr_scheduler_ctor=None,
                    collate_fn=custom_collate_fn):
    """
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0)
    
    if lr_scheduler_ctor is not None:
        lr_scheduler = lr_scheduler_ctor(optimizer)
    else:
        lr_scheduler = None

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    best_val_loss = float('inf')
    best_epoch_i = 0
    best_model = copy.deepcopy(model)

    for epoch_i in range(epoch_n):
        try:
            epoch_start = datetime.datetime.now()
            print('Эпоха {}'.format(epoch_i))

            model.train()
            mean_train_loss = 0
            train_batches_n = 0
            for batch_i, (word_ids, char_ids, tag_ids, _) in enumerate(train_dataloader):
                if batch_i > max_batches_per_epoch_train:
                    break

                word_ids = copy_data_to_device(word_ids, device)
                char_ids = copy_data_to_device(char_ids, device)
                tags = copy_data_to_device(tag_ids, device)

                pred = model(word_ids, char_ids)
                loss = criterion(pred.view(-1, len(tag_vocab)), tags.view(-1))

                model.zero_grad()
                loss.backward()

                optimizer.step()

                mean_train_loss += float(loss)
                train_batches_n += 1

            mean_train_loss /= train_batches_n
            print('Эпоха: {} итераций, {:0.2f} сек'.format(train_batches_n,
                                                           (datetime.datetime.now() - epoch_start).total_seconds()))
            print('Среднее значение функции потерь на обучении', mean_train_loss)


            model.eval()
            mean_val_loss = 0
            val_batches_n = 0

            with torch.no_grad():
                for batch_i, (word_ids, char_ids, tag_ids, _) in enumerate(val_dataloader):
                    if batch_i > max_batches_per_epoch_val:
                        break

                    word_ids = copy_data_to_device(word_ids, device)
                    char_ids = copy_data_to_device(char_ids, device)
                    tags = copy_data_to_device(tag_ids, device)

                    pred = model(word_ids, char_ids)
                    loss = criterion(pred.view(-1, len(tag_vocab)), tags.view(-1))

                    mean_val_loss += float(loss)
                    val_batches_n += 1

            mean_val_loss /= val_batches_n
            print('Среднее значение функции потерь на валидации', mean_val_loss)

            if mean_val_loss < best_val_loss:
                best_epoch_i = epoch_i
                best_val_loss = mean_val_loss
                best_model = copy.deepcopy(model)
                print('Новая лучшая модель!')
            elif epoch_i - best_epoch_i > early_stopping_patience:
                print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(
                    early_stopping_patience))
                break

            if lr_scheduler is not None:
                lr_scheduler.step(mean_val_loss)

            print()
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
        except Exception as ex:
            print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
            break

    return best_val_loss, best_model

In [37]:
from torch.utils.data import random_split

train_size = int(0.7 * len(documents))
val_size = int(0.15 * len(documents))
test_size = len(documents) - train_size - val_size

#создание датасета
dataset = NERDataset(documents, word_vocab, char_vocab, tag_vocab)
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

def run_model(word_emb_dim, char_emb_dim, hidden_dim, bi, concat): 
    batch_size=64
    lstm_model = LSTM_NER(vocab_size=len(word_vocab), char_vocab_size=len(char_vocab) * batch_size, num_tags=len(tag_vocab), 
                          word_embed_dim=word_emb_dim, char_embed_dim=char_emb_dim, hidden_dim=hidden_dim, bi=bi, concat=concat)
    
    (best_val_loss, best_model) = train_eval_loop(lstm_model,
                                                    train_dataset,
                                                    val_dataset,
                                                    F.cross_entropy,
                                                    lr=5e-3,
                                                    epoch_n=5,
                                                    device='cpu',
                                                    batch_size=batch_size,
                                                    early_stopping_patience=2,
                                                    max_batches_per_epoch_train=500,
                                                    max_batches_per_epoch_val=100,
                                                    lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                        factor=0.5)) #,verbose=True

    return {'loss': best_val_loss, 'model': best_model}


In [38]:
word_emb_dim=300
char_emb_dim=100
hidden_dim=300

print('LSTM model with concat')
lstm_concat = run_model(word_emb_dim=word_emb_dim, char_emb_dim=char_emb_dim, hidden_dim=hidden_dim, bi=False, concat=True)

print()
print('BiLSTM model with concat')
bilstm_concat = run_model(word_emb_dim=word_emb_dim, char_emb_dim=char_emb_dim, hidden_dim=hidden_dim, bi=True, concat=True)

print()
print('LSTM model with sum')
lstm_sum = run_model(word_emb_dim=word_emb_dim, char_emb_dim=char_emb_dim, hidden_dim=hidden_dim, bi=False, concat=False)

print()
print('BiLSTM model with sum')
bilstm_sum = run_model(word_emb_dim=word_emb_dim, char_emb_dim=char_emb_dim, hidden_dim=hidden_dim, bi=True, concat=False)

best = min([lstm_concat, lstm_sum, bilstm_concat, bilstm_sum], key=lambda item: item['loss'])
best_model = best['model']
best

LSTM model with concat
Эпоха 0
Эпоха: 11 итераций, 58.02 сек
Среднее значение функции потерь на обучении 0.5214072167873383
Среднее значение функции потерь на валидации 0.4249001145362854
Новая лучшая модель!

Эпоха 1
Эпоха: 11 итераций, 69.02 сек
Среднее значение функции потерь на обучении 0.4077836193821647
Среднее значение функции потерь на валидации 0.3985764781634013
Новая лучшая модель!

Эпоха 2
Эпоха: 11 итераций, 67.43 сек
Среднее значение функции потерь на обучении 0.35815347053787927
Среднее значение функции потерь на валидации 0.38801873723665875
Новая лучшая модель!

Эпоха 3
Эпоха: 11 итераций, 68.53 сек
Среднее значение функции потерь на обучении 0.30691819570281287
Среднее значение функции потерь на валидации 0.3850449323654175
Новая лучшая модель!

Эпоха 4
Эпоха: 11 итераций, 54.70 сек
Среднее значение функции потерь на обучении 0.26072522726925934
Среднее значение функции потерь на валидации 0.40136730670928955


BiLSTM model with concat
Эпоха 0
Эпоха: 11 итераций, 100.

{'loss': 0.3802920877933502,
 'model': LSTM_NER(
   (word_embed): Embedding(44160, 300)
   (char_embed): Embedding(10880, 100)
   (char_lstm): LSTM(100, 150, bidirectional=True)
   (lstm): LSTM(300, 300, bidirectional=True)
   (linear): Linear(in_features=600, out_features=300, bias=True)
   (classifier): Linear(in_features=600, out_features=3, bias=True)
 )}

Модели быстро переобучаются (на валидационной выборке начинает расти фунция потерь, а на тренировочной падает), поэтому как правило 4 эпохи достаточно. 
Остальные параметры подобраны, по лучшим показателям.

Реализация BiLSTM https://docs.pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

#### метрики

In [39]:
def predict_with_model(model, dataset, device=None, batch_size=64, num_workers=0, return_labels=False):
    """
    :param model: torch.nn.Module - обученная модель
    :param dataset: torch.utils.data.Dataset - данные для применения модели
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :return: numpy.array размерности len(dataset) x *
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    results_by_batch = []

    device = torch.device(device)
    model.to(device)
    model.eval()

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=custom_collate_fn)
    labels = []
    with torch.no_grad():
        for word_ids, char_ids, tag_ids, _ in dataloader:
            word_ids = copy_data_to_device(word_ids, device)
            char_ids = copy_data_to_device(char_ids, device)
            tags = copy_data_to_device(tag_ids, device)

            if return_labels:
                labels.append(tag_ids.numpy())

            batch_pred = model(word_ids, char_ids)
            results_by_batch.append(batch_pred.detach().cpu().numpy())

    if return_labels:
        return np.concatenate(results_by_batch, 0), np.concatenate(labels, 0)
    else:
        return np.concatenate(results_by_batch, 0)

In [40]:
train_pred, train_labels = predict_with_model(best_model, train_dataset, return_labels=True)
train_loss = F.cross_entropy(torch.tensor(train_pred).view(-1, len(tag_vocab)),
                             torch.tensor(train_labels).view(-1))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.reshape(-1), train_pred.argmax(2).reshape(-1), target_names=target_names))
print()

test_pred, test_labels = predict_with_model(best_model, test_dataset, return_labels=True)
test_loss = F.cross_entropy(torch.tensor(test_pred).view(-1, len(tag_vocab)),
                            torch.tensor(test_labels).view(-1))
print('Среднее значение функции потерь на тесте', float(test_loss))
print(classification_report(test_labels.reshape(-1), test_pred.argmax(2).reshape(-1), target_names=target_names))

Среднее значение функции потерь на обучении 0.2727467119693756
              precision    recall  f1-score   support

           O       0.92      0.99      0.95     78694
       B-PER       0.64      0.42      0.51      6978
       I-PER       0.84      0.02      0.03      3928

    accuracy                           0.90     89600
   macro avg       0.80      0.48      0.50     89600
weighted avg       0.89      0.90      0.88     89600


Среднее значение функции потерь на тесте 0.36165136098861694
              precision    recall  f1-score   support

           O       0.91      0.98      0.94     17028
       B-PER       0.48      0.30      0.37      1404
       I-PER       0.50      0.01      0.02       768

    accuracy                           0.89     19200
   macro avg       0.63      0.43      0.44     19200
weighted avg       0.86      0.89      0.86     19200



Так как у нас нет нормального распределения, то accuracy будет завышено и неинформативно. Поэтому смотрим на строчку macro avg. Видно, что модель недообучена, данных для обучения не достаточно. Так, например, для I-PER показатели f1-score совсем маленькие, их модель почти не находит. Для B-PER получше, но очевидно не достаточно хорошо.

### Сравнение с библиотекой natasha 

In [41]:

from natasha import Doc, Segmenter, NewsEmbedding, NewsNERTagger
from IPython.display import display, HTML

def show_comparison(text, our_tags, natasha_tags):
    """визуальное сравнение результатов нашей модели и natasha
    
    параметры:
        text: исходный текст
        our_tags: теги, предсказанные нашей моделью
        natasha_tags: теги от natasha
    """
    our_html = []
    natasha_html = []
    
    # формируем html с подсветкой сущностей
    for token, our_tag, natasha_tag in zip(text.split(), our_tags, natasha_tags):
        # для нашей модели
        if our_tag != 'O':
            our_html.append(f"<mark style='background:#ffcccc'>{token}({our_tag})</mark>")
        else:
            our_html.append(token)
            
        # для natasha
        if natasha_tag != 'O':
            natasha_html.append(f"<mark style='background:#ccffcc'>{token}({natasha_tag})</mark>")
        else:
            natasha_html.append(token)
    
    # выводим сравнение в две колонки
    display(HTML(f"""
    <div style="display:flex">
        <div style="flex:50%; padding:10px">
            <h4>наша модель</h4>
            {' '.join(our_html[:128])}...
        </div>
        <div style="flex:50%; padding:10px">
            <h4>natasha</h4>
            {' '.join(natasha_html[:128])}...
        </div>
    </div>
    """))

In [42]:
segmenter = Segmenter()
ner_tagger = NewsNERTagger(NewsEmbedding())

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
best_model.to(device)

test_data = []

# Проверяем каждое 10е предложение
for text_i in range(0, len(test_dataset), 10):
    data = test_dataset[text_i]
    text = data['text']
    tokens = data['tokens']
    
    # Natasha 
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    
    natasha_tags = ['O'] * len(tokens)
    for span in doc.spans:
        if span.type == 'PER':
            span_tokens = span.text.split()
            for j, token in enumerate(tokens):
                if token == span_tokens[0] and j + len(span_tokens) <= len(tokens):
                    if ' '.join(tokens[j:j+len(span_tokens)]) == span.text:
                        natasha_tags[j] = 'B-PER'
                        for k in range(1, len(span_tokens)):
                            natasha_tags[j+k] = 'I-PER'
                        break
    
    best_model.eval()
    with torch.no_grad():
        word_ids = copy_data_to_device(data['word_ids'].unsqueeze(0), device)
        char_ids = copy_data_to_device(data['char_ids'].unsqueeze(0), device)
        output = best_model(word_ids, char_ids)
        pred_ids = torch.argmax(output, dim=-1).squeeze(0).cpu().tolist()
        best_model_tags = [id_to_tag[i] for i in pred_ids[:len(tokens)]]
    
    test_data.append({'text': text, 'tokens': tokens, 'annotations': data['annotations'], 'best_model_tags': best_model_tags, 'natasha_tags': natasha_tags})
    
    show_comparison(text, best_model_tags, natasha_tags)

In [43]:
def model_ners(text, tokens, pred_tags):       
    model_entities = []
    pos = 0
    for j, token in enumerate(tokens):
        token_start = text.find(token, pos)
        if token_start == -1:
            continue
        token_end = token_start + len(token)
        pos = token_end

        if pred_tags[j] == 'B-PER':
            model_entities.append({'start': token_start, 'end': token_end, 'text': token})
        elif pred_tags[j] == 'I-PER':
            current_entity = model_entities[len(model_entities) - 1] if len(model_entities) > 0 else None
            if current_entity and current_entity['end'] + 1 == token_start:
                current_entity['end'] = token_end
                current_entity['text'] += ' ' + token
    return model_entities

def count_metrics(annotations, preds):
    exact = partial = missing = 0
    for ann in annotations:
        found = '' 
        for pred in preds:
            if pred['start'] == ann['offset'] and pred['end'] == ann['offset'] + ann['length']:
                found = 'exact'
                break
            if pred['start'] < ann['offset'] + ann['length'] and pred['end'] > ann['offset']:
                found = 'partial'
                break
        exact += 1 if found == 'exact' else 0
        partial += 1 if found == 'partial' else 0
        missing += 1 if found == '' else 0
    return {'exact': exact, 'partial': partial, 'missing': missing}

metrics_ret = []

for data in test_data:
    bm_tags = data['best_model_tags']
    n_tags = data['natasha_tags']
    tokens = data['tokens']
    text = data['text']
    annotations = data['annotations']

    bm_ners = model_ners(text, tokens, bm_tags)
    n_ners = model_ners(text, tokens, n_tags)

    bm_metrics = count_metrics(annotations, bm_ners)
    n_metrics = count_metrics(annotations, n_ners)

    metrics_ret.append({'best_model': bm_metrics, 'natasha': n_metrics, 'all': len(annotations)})

metrics_result = {'best_model': 
                    {'exact': sum([bm['best_model']['exact'] for bm in metrics_ret]), 
                     'partial': sum([bm['best_model']['partial'] for bm in metrics_ret]), 
                     'missing': sum([bm['best_model']['missing'] for bm in metrics_ret])},
                  'natasha': 
                    {'exact': sum([bm['natasha']['exact'] for bm in metrics_ret]), 
                     'partial': sum([bm['natasha']['partial'] for bm in metrics_ret]), 
                     'missing': sum([bm['natasha']['missing'] for bm in metrics_ret])}, 
                  'all': sum([m['all'] for m in metrics_ret])}

metrics_result
    

{'best_model': {'exact': 0, 'partial': 19, 'missing': 155},
 'natasha': {'exact': 7, 'partial': 43, 'missing': 124},
 'all': 174}

# Вывод

Наша лучшая модель находит очень мало сущностей и точных и частичных, много пропустила, модель Natasha — получше, но тоже много было пропущено.

Наша лучшая модель по всем показателям работает хуже предобученной Natasha.